In [ ]:
from importlib import reload
from os.path import join
from collections import defaultdict
import sys
import numpy as np
import cv2
cv2.setNumThreads(1)
import os

In [ ]:
import ipywidgets
from IPython.display import display, clear_output
from matplotlib import pyplot

In [ ]:
from datasets.dshdf5pose import Hdf5PoseDataset
import datatransformation
import neuralnets.models as models
import vis
import utils
import train
from datasets.infinitelyrepeatinginterleaveddatasets import InfinitelyRepeatingInterleavedDatasets

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as tf
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset, ConcatDataset
from torchvision import transforms

In [ ]:
inputsize = 129
datadir = os.environ['DATADIR']

augment = [
    datatransformation.ApplyRoiRandomized(),
    datatransformation.Rescale((inputsize,inputsize)),
    datatransformation.AdaptiveBrightnessContrastDistortion(),
    datatransformation.Flip(),
]
testpreprocess = [
    datatransformation.ApplyRoi(),
    datatransformation.Rescale(inputsize+1),
    datatransformation.CenterCrop(inputsize),
]
normalize_and_tensor = [
    datatransformation.Normalize(monochrome=True),
    datatransformation.ToTensor()
]

ds_train_aflw = Hdf5PoseDataset(join(datadir,'aflw2k.h5'), shuffle=True, subset=slice(400,None), transform=transforms.Compose([
    datatransformation.InjectPt3d68Enable(),
    datatransformation.InjectPoseEnable(),
    *augment,
    *normalize_and_tensor
]))

ds_full_300wlp = Hdf5PoseDataset(join(datadir,'300wlp.h5'), shuffle=True, transform=transforms.Compose([
    datatransformation.InjectPt3d68Enable(),
    datatransformation.InjectPoseEnable(),
    *augment,
    *normalize_and_tensor
]))

ds_full_ytfaces = Hdf5PoseDataset(join(datadir,'ytfaces.h5'), shuffle=True, transform=transforms.Compose([
    datatransformation.InjectPoseEnable(),
    datatransformation.InjectPt3d68Enable(),
    *augment,
    *normalize_and_tensor
]))


# ds_test_biwi = Hdf5PoseDataset(join(datadir,'biwi.h5'), shuffle=True, subset=None, transform=transforms.Compose([
#     datatransformation.InjectZeroKeypoints3d(),
#     datatransformation.InjectPoseEnable(),
#     *testpreprocess,
#     *normalize_and_tensor
# ]))

ds_test_aflw = Hdf5PoseDataset(join(datadir,'aflw2k.h5'), shuffle=False, subset=slice(400), transform=transforms.Compose([
    datatransformation.InjectPt3d68Enable(),
    datatransformation.InjectPoseEnable(),
    *testpreprocess,
    *normalize_and_tensor
]))

In [ ]:
print ("Train ds sizes: ", [len(ds) for ds in [ds_train_aflw, ds_full_300wlp, ds_full_ytfaces]])

In [ ]:
ds_test = ds_test_aflw

#ds_train = ds_train_aflw
ds_train = InfinitelyRepeatingInterleavedDatasets(
    32*1024, 
    [ds_train_aflw, ds_full_ytfaces, ds_full_300wlp], 
    [1., 10., 10.])

train_loader = datatransformation.PostprocessingDataLoader(ds_train, 
                          batch_size=128,
                          shuffle=False, 
                          num_workers=5,
                          postprocess = transforms.Compose([
                                datatransformation.BlurNoiseDistortion(),
                          ]))
test_loader = DataLoader(ds_test, 
                          batch_size=128,
                          shuffle=False, 
                          num_workers=5)

In [ ]:
net = models.NetworkWithPointHead()
net.cuda()

In [ ]:
all_params = net.parameters()
backbone_params_1 = net.convnet.backbone[:9].parameters()
backbone_params_2 = net.convnet.backbone[9:11].parameters()
backbone_params_3 = net.convnet.backbone[14:].parameters()
untrained_params = list(set(all_params) - set(backbone_params_1) - set(backbone_params_2) - set(backbone_params_3))

In [ ]:
%matplotlib inline
pyplot.ioff()

#optimizer = optim.Adam(net.parameters(), lr=1.e-3)
optimizer = optim.Adam([
    {'params': backbone_params_1, 'lr': 1e-6},
    {'params': backbone_params_2, 'lr': 1e-4},
    {'params': backbone_params_3, 'lr': 1e-3},
    {'params': untrained_params}],
    lr=1.e-3)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [40, 80], gamma=0.33)

network_filename = None

n_epochs = 100

criterions = [
    train.MixedParameterKeypointLoss(n_epochs),
    #train.AllParameterLoss(),
    #train.Points3dLoss(),
    train.BoxLoss(),
    train.QuaternionNormalizationRegularization(),
]
critweights = np.array([1., 0.1, 1.e-5 ])

train.run_the_training(n_epochs,
                      optimizer,
                      net,
                      train_loader,
                      test_loader,
                      criterions,
                      critweights,
                      2,
                      join('..','model_files'),
                      network_filename,
                      other_metrics = [
                        train.QuatPoseLoss2(),
                        train.CoordPoseLoss(),
                        train.ShapeParameterLoss(),
                        train.Points3dLoss(),
                      ],
                      scheduler = scheduler)